This section imports the original Excel contact sheet and prepares the data for analysis. It first extracts the actual column names from the first row, then standardizes all column headers by converting them to lowercase and replacing spaces with underscores. A typo in the column name `first_outeach` is corrected to `first_outreach`, which is then converted into datetime format for later time-based analysis. These cleaning steps ensure consistency in the dataset and lay the groundwork for identifying ideal outreach candidates and creating interactive visualizations in subsequent sections.

In [2]:
import pandas as pd

# 读取 Excel 文件（假设文件放在 data 文件夹中）
file_path = "../data/Finance Dep. Contact sheet.xlsx"

# 载入原始 Excel 文件
raw_df = pd.read_excel(file_path, sheet_name=0)

# 取第 1 行作为正式列名（跳过前导解释行）
df = raw_df.copy()
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# 标准化列名（可选）
df.columns.name = None
df.columns = [str(c).strip().lower().replace(' ', '_') for c in df.columns]

# 日期处理：转换 first_outreach 列为 datetime 类型
df = df.rename(columns={
    'first_outeach': 'first_outreach'
})

df['first_outreach'] = pd.to_datetime(df['first_outreach'], errors='coerce', dayfirst=True)

# 补充字段（布尔）
df['is_contacted'] = df['current_status'].str.lower().fillna('').str.contains('outreach')
df['has_contact_info'] = df['contact_information'].notna()
df['is_english'] = df['language'].str.lower() == 'english'

# 示例：自定义“理想联系人”筛选函数
def is_ideal(row):
    donation_type = str(row.get('type_of_donation', '')).lower()
    return (
        row.get('type', '').lower() == 'non profit' and
        'financial' in donation_type or 'material' in donation_type and
        not row.get('is_contacted', False) and
        row.get('has_contact_info', False)
    )

df['is_ideal_candidate'] = df.apply(is_ideal, axis=1)

# 显示结果预览（可选）
df.head()


,name,contact_information,location,type,outreach_type,responsible,current_status,first_outreach,follow_up,meeting,...,note,nan,nan,email_types,language,email_sample,is_contacted,has_contact_info,is_english,is_ideal_candidate
0,Action for Nature Youth Eco-Hero Award,Website,Great Britain,small grants,Type 7: Grant,Niclas K.,first outreach,2025-07-11,NaN,NaN,...,Only for people under 16...,NaN,NaN,NaN,NaN,NaN,True,True,False,False
1,AIMI Kids,partnership@amikids.org.,USA,Non profit,Type 2: material donation,TBD PA,first outreach,2025-07-21,NaN,NaN,...,NaN,NaN,NaN,Werft 1,English,see here,True,True,True,True
2,Airbnb,contact.press@airbnb.com,Usa,for profit,Type 3: financial donation,TBD,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,Mat 2,English,see here,False,True,True,False
3,Aldi Süd - Regionalgesellschaft Mühldorf/Karls...,NaN,Germany,for profit,Type 6: Finanzielle Spende,TBD,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,finance 3,English,see here,False,False,True,False
4,Allbirds (shoes),NaN,Usa,for profit,Type 3: financial donation,TBD,NaN,NaT,NaN,NaN,...,NaN,NaN,NaN,Werft 1,German,see here,False,False,False,False
